In [1]:
#| label: irFig2jn

from repo2data.repo2data import Repo2Data
import os 
import pickle
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML
from plotly import tools

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    data_req_path = os.path.join("..","..","..", "binder", "data_requirement.json")
    repo2data = Repo2Data(data_req_path)
    DATA_ROOT = os.path.join(repo2data.install()[0],"t1-book-neurolibre")
    filename = os.path.join(DATA_ROOT,"01",'figure_2.pkl')

with open(filename, 'rb') as f:
    params, signal_WM, signal_GM, signal_CSF = pickle.load(f)

config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

wm = go.Scatter(
    x = params["TI"],
    y = signal_WM,
    name = 'T<sub>1</sub> = 0.9 s (White Matter)',
    text = 'T<sub>1</sub> = 0.9 s (White Matter)',
    hoverinfo = 'x+y+text'
)

gm = go.Scatter(
    x = params["TI"],
    y = signal_GM,
    name = 'T<sub>1</sub> = 1.5 s (Grey Matter)',
    text = 'T<sub>1</sub> = 1.5 s (Grey Matter)',
    hoverinfo = 'x+y+text'
)

csf = go.Scatter(
    x = params["TI"],
    y = signal_CSF,
    name = 'T<sub>1</sub> = 4.0 s (Cerebrospinal Fluid)',
    text = 'T<sub>1</sub> = 4.0 s (Cerebrospinal Fluid)',
    hoverinfo = 'x+y+text'
)

data = [wm, gm, csf]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=0,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.175,
            showarrow=False,
            text='Inversion Time – TI (ms)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.50,
            showarrow=False,
            text='Long. Magnetization (M<sub>z</sub>)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.55,
        y=0.15,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    plot_bgcolor='white'
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'ir_fig_2.html', config = config)

#display(HTML('ir_fig_2.html'))